## FIT5120: IE Studio experience project
### Project: Link2 HerResilience

Team: Harmony (TA18)
Author: Mandeep Singh

Data wrangling process for epic6: yoga dataset

In [1]:
import os
import csv


In [2]:
path_dir = os.getcwd() + "/Yoga Postures Dataset"
path_images_dir = path_dir + "/Images"

list_yoga_asanas = [x for x in os.listdir(path_images_dir)]
list_yoga_asanas

['Anantasana',
 'Ardhakati Chakrasana',
 '.DS_Store',
 'Vajrasana',
 'Tadasana',
 'Marjariasana',
 'Parvatasana',
 'Bhujangasana',
 'Viparita Karani',
 'Kati Chakrasana',
 'Sarvangasana']

### Pre-processing: Extract & transform

In [3]:
import re

dict_yoga_asanas_wrong_postures = {}
for asana in list_yoga_asanas:
    if not asana.lower() == ".ds_store":
        list_wrong_postures_in_sequence = os.listdir(path_images_dir + "/" + asana + "/" + asana + " Wrong Steps")
        
        # dict of wrong_postures in a step
        dict_wrong_postures_in_step = {}
        for s in list_wrong_postures_in_sequence:
            # print(s)
            # extract asana step number from s
            if not s.lower() == ".ds_store":
                step = int(re.search(r"Step (\d)", s)[1])
                
                # extract posture
                posture = re.search(r"\d- ?([A-Za-z ()]*)", s)[1]#.lower()
                
                if not step in dict_wrong_postures_in_step.keys():
                    dict_wrong_postures_in_step[step] = [posture]
                else:
                    dict_wrong_postures_in_step[step].append(posture)
        
        dict_yoga_asanas_wrong_postures[asana] = dict_wrong_postures_in_step
    
dict_yoga_asanas_wrong_postures


{'Anantasana': {2: ['Knee'], 1: ['Legs', 'Neck'], 3: ['Knee'], 4: ['Hand']},
 'Ardhakati Chakrasana': {2: ['Ankle', 'Head'],
  3: ['Knee', 'Legs'],
  5: ['Knee', 'Legs'],
  4: ['Legs'],
  1: ['Head', 'Palms']},
 'Vajrasana': {2: ['Back and Head'], 1: ['Legs and Hand']},
 'Tadasana': {1: ['Legs'],
  2: ['Legs and Hand'],
  5: ['Standing'],
  3: ['Legs and Hand'],
  4: ['Hands and Legs']},
 'Marjariasana': {2: ['Legs'],
  3: ['Legs'],
  4: ['Hand and Legs'],
  1: ['Hand and Neck']},
 'Parvatasana': {3: ['Leg and Hand'],
  2: ['Leg and Hand', 'Hand'],
  1: ['Leg and Hand'],
  4: ['Hand']},
 'Bhujangasana': {2: ['Legs', 'Hand and Legs'],
  1: ['Head', 'Legs'],
  3: ['Legs', 'Head']},
 'Viparita Karani': {1: ['Legs', 'Neck'],
  2: ['Legs', 'Knees', 'Neck', 'Back (Bending)']},
 'Kati Chakrasana': {2: ['Hand'], 4: ['Neck'], 1: ['Legs'], 3: ['Neck']},
 'Sarvangasana': {1: ['Neck and Hand', 'Neck', 'Legs'],
  2: ['Legs', 'Back (Overbend)', 'Knees']}}

In [4]:
# add 2 more as per functionality
dict_yoga_asanas_wrong_postures['Pranayama'] = {1: ['Legs', 'Land'], 2: ['Land'], 3: ['Hand']}
dict_yoga_asanas_wrong_postures['Chakrasana'] = {1: ['Legs', 'Back'], 2: ['Legs', 'Back', 'Hand'], 
                                                 3: ['Legs', 'Back', 'Hips'], 4: ['Legs', 'Back']}


In [5]:
import json

# save in json
with open(path_dir + "/asanas_wrong_postures_in_step.json", "w") as file:
    json.dump(dict_yoga_asanas_wrong_postures, file)



### Transformation to pose estimation vectors

In [6]:
import track
from os.path import join, normpath
from pymediainfo import MediaInfo
import numpy as np
import uuid
import csv
import time

def transform_pose_estimate(file_path="", model_name="II", framework="pytorch", 
                            b_visualize=True, b_store=True, b_verbose=True):
    if 'Image' in [track.track_type for track in MediaInfo.parse(file_path).tracks]:
        b_tracked = track.perform_tracking(video=False, file_path=normpath(file_path), 
                                model_name=model_name, framework_name=framework, 
                                visualize=b_visualize, store=b_store, b_verbose=b_verbose)

        # if b_visualize and b_tracked:
        #     filename = file_name.split(".")[0]+"_tracked.png"
            
        return b_tracked#, coordinates

    else:
        return None
    

In [7]:
import glob

def transform_images_dataset(img_dir_path):
    for f in glob.glob(img_dir_path + "/**/*.jpg", recursive=True):
        # print(f)
        b_tracked = transform_pose_estimate(file_path=f, model_name="II", 
                                                         framework="pytorch", b_visualize=False, 
                                                         b_store=True, b_verbose=False)

    

In [ ]:
transform_images_dataset(path_images_dir)

/Users/mandeep/py_venv/lib/python3.9/site-packages/torch/serialization.py:1113: SourceChangeWarning: source code of class 'MainModel.KitModel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/mandeep/py_venv/lib/python3.9/site-packages/torch/serialization.py:1113: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/mandeep/py_venv/lib/python3.9/site-packages/torch/serialization.py:1113: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by accessing the object'


##########################################################################################################
Image processed in 2.174 seconds
##########################################################################################################


##########################################################################################################
Image processed in 2.541 seconds
##########################################################################################################


##########################################################################################################
Image processed in 2.063 seconds
##########################################################################################################


##########################################################################################################
Image processed in 2.054 seconds
##########################################################################################################


###

In [8]:
import csv
import pandas as pd

def combine_coordinates(path_dir):
    '''
    method to combine all pose estimates in coordinates.csv of each image in dir
    '''
    for i, f in enumerate(glob.glob(path_dir + "/*.csv", recursive=True)):
        df_csvrows = pd.read_csv(f)
        if i == 0:
            df_csvrows.to_csv(path_dir + "/coordinates.csv")                   
        else:
            # combine csv
            df_csvrows.to_csv(path_dir + "/coordinates.csv", mode='a')
            

# enrich coordinates data for each steps
def transform_images_dataset(img_dir_path):
    for asana, postures_seq in dict_yoga_asanas_wrong_postures.items():
        for step, postures in postures_seq.items():
            # dir path Right Steps
            path_right_step = path_images_dir + "/" + asana + "/" + asana + " Right Steps/" +\
                             asana + " Right Step " + str(step)
            # combine all coordinates vec
            combine_coordinates(path_right_step)

            for posture in postures:
                # dir path Wrong Steps
                path_wrong_step = path_images_dir + "/" + asana + "/" + asana + " Wrong Steps/" +\
                                 asana + " Wrong Step " + str(step) + "- " + posture
                if not os.path.isdir(path_wrong_step):
                    path_wrong_step = path_images_dir + "/" + asana + "/" + asana + " Wrong Steps/" +\
                                 asana + " Wrong Step " + str(step) + "-" + posture
                # combine all coordinates vec
                combine_coordinates(path_wrong_step)
            
                        
        


In [9]:
transform_images_dataset(path_images_dir)


In [21]:
# class labels count
for asana, postures_seq in dict_yoga_asanas_wrong_postures.items():
    if asana in {"Pranayama", "Chakrasana"}:
        continue
    # dict_knn_clf[asana] = []
    for step, postures in postures_seq.items():
        # dir path Right Steps
        path_right_step = path_images_dir + "/" + asana + "/" + asana + " Right Steps/" +\
                         asana + " Right Step " + str(step)
        # READ combined coordinates vec
        df = pd.read_csv(path_right_step + "/coordinates.csv")
        print(df.head(5))
        break

   Unnamed: 0          head_top_x          head_top_y        upper_neck_x  \
0         0.0  0.9089673913043478  0.5203804347826086  0.7622282608695652   
1         NaN          head_top_x          head_top_y        upper_neck_x   
2         0.0  0.9252717391304348  0.5122282608695652  0.7785326086956522   
3         NaN          head_top_x          head_top_y        upper_neck_x   
4         0.0  0.7241847826086957  0.6644021739130435  0.5855978260869565   

         upper_neck_y    right_shoulder_x    right_shoulder_y  \
0  0.4741847826086957  0.7839673913043478  0.4442934782608695   
1        upper_neck_y    right_shoulder_x    right_shoulder_y   
2  0.4796195652173913  0.8002717391304348  0.4877717391304347   
3        upper_neck_y    right_shoulder_x    right_shoulder_y   
4  0.6209239130434783  0.5964673913043478  0.6290760869565217   

        right_elbow_x       right_elbow_y       right_wrist_x  ...  \
0             0.78125  0.3845108695652174  0.7323369565217391  ...   
1     

### kNN classifier

In [33]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import pickle

def build_knn_classifiers(img_dir_path, k=3):
    # # dict of knn classifier
    # dict_knn_clf = {}
    for asana, postures_seq in dict_yoga_asanas_wrong_postures.items():
        if asana in {"Pranayama", "Chakrasana"}:
            continue
        # dict_knn_clf[asana] = []
        for step, postures in postures_seq.items():
            # dir path Right Steps
            path_right_step = path_images_dir + "/" + asana + "/" + asana + " Right Steps/" +\
                             asana + " Right Step " + str(step)
            # READ combined coordinates vec
            df = pd.read_csv(path_right_step + "/coordinates.csv")
            df.drop(columns="Unnamed: 0", inplace=True)
            df.drop_duplicates(inplace=True)
            df.drop(index=1, inplace=True)
            
            # class label 
            Y = np.zeros(df.shape[0])
            # print(asana, step, df.shape, Y.shape, df.columns)
            # print(df.head())
            # print(df.to_numpy().shape)

            for i, posture in enumerate(postures):
                # dir path Wrong Steps
                path_wrong_step = path_images_dir + "/" + asana + "/" + asana + " Wrong Steps/" +\
                                 asana + " Wrong Step " + str(step) + "- " + posture
                if not os.path.isdir(path_wrong_step):
                    path_wrong_step = path_images_dir + "/" + asana + "/" + asana + " Wrong Steps/" +\
                                 asana + " Wrong Step " + str(step) + "-" + posture
                # read combined coordinates vec
                curr_df = pd.read_csv(path_wrong_step + "/coordinates.csv")
                curr_df.drop(columns="Unnamed: 0", inplace=True)
                curr_df.drop_duplicates(inplace=True)
                curr_df.drop(index=1, inplace=True)
                curr_Y = np.ones(curr_df.shape[0]) * (i+1)

                # concat df
                df = pd.concat([df, curr_df])
                Y = np.concatenate((Y, curr_Y))
                
                # print(asana, step, df.shape, Y.shape)
                
            
            X_train, X_test, y_train, y_test = train_test_split(df.to_numpy(), Y, 
                                                                train_size= 0.8, shuffle=True)#, stratify=)
            
            knn_clf = KNeighborsClassifier(n_neighbors=k)
            knn_clf.fit(X_train, y_train)

            score = knn_clf.score(X_test, y_test)
            print(f"{asana} step#{step} knn_clf score: {score}")

            # save the clf
            pkl_path = path_images_dir + "/knn_clf-" + asana + "_" + str(step) + ".pkl"
            pickle.dump(knn_clf, open(pkl_path, 'wb'))


In [36]:
# build knn_classifiers
build_knn_classifiers(path_images_dir)


Anantasana step#2 knn_clf score: 0.8367346938775511
Anantasana step#1 knn_clf score: 0.8095238095238095
Anantasana step#3 knn_clf score: 0.7428571428571429
Anantasana step#4 knn_clf score: 0.9230769230769231
Ardhakati Chakrasana step#2 knn_clf score: 0.8
Ardhakati Chakrasana step#3 knn_clf score: 0.6551724137931034
Ardhakati Chakrasana step#5 knn_clf score: 0.6071428571428571
Ardhakati Chakrasana step#4 knn_clf score: 0.76
Ardhakati Chakrasana step#1 knn_clf score: 0.7868852459016393
Vajrasana step#2 knn_clf score: 0.6666666666666666
Vajrasana step#1 knn_clf score: 0.5384615384615384
Tadasana step#1 knn_clf score: 0.9264705882352942
Tadasana step#2 knn_clf score: 0.717391304347826
Tadasana step#5 knn_clf score: 0.8529411764705882
Tadasana step#3 knn_clf score: 0.9411764705882353
Tadasana step#4 knn_clf score: 0.7391304347826086
Marjariasana step#2 knn_clf score: 0.7272727272727273
Marjariasana step#3 knn_clf score: 0.7894736842105263
Marjariasana step#4 knn_clf score: 0.745454545454545

In [18]:
# build knn_classifiers k=4
build_knn_classifiers(path_images_dir, k=4)


Anantasana step#2 knn_clf score: 0.673469387755102
Anantasana step#1 knn_clf score: 0.8333333333333334
Anantasana step#3 knn_clf score: 0.8285714285714286
Anantasana step#4 knn_clf score: 0.7692307692307693
Ardhakati Chakrasana step#2 knn_clf score: 0.775
Ardhakati Chakrasana step#3 knn_clf score: 0.4827586206896552
Ardhakati Chakrasana step#5 knn_clf score: 0.5714285714285714
Ardhakati Chakrasana step#4 knn_clf score: 0.84
Ardhakati Chakrasana step#1 knn_clf score: 0.7049180327868853
Vajrasana step#2 knn_clf score: 0.6666666666666666
Vajrasana step#1 knn_clf score: 0.38461538461538464
Tadasana step#1 knn_clf score: 0.9264705882352942
Tadasana step#2 knn_clf score: 0.717391304347826
Tadasana step#5 knn_clf score: 0.8970588235294118
Tadasana step#3 knn_clf score: 0.8823529411764706
Tadasana step#4 knn_clf score: 0.6739130434782609
Marjariasana step#2 knn_clf score: 0.6590909090909091
Marjariasana step#3 knn_clf score: 0.7894736842105263
Marjariasana step#4 knn_clf score: 0.7818181818181